In [18]:
#!pip3 install bayesian-optimization==0.6 --user
#!pip3 install pip install bayesian-optimization --user
!pip3 install git+https://github.com/fmfn/BayesianOptimization

  Running command git clone --filter=blob:none --quiet https://github.com/fmfn/BayesianOptimization 'C:\Users\Cyberpower PC\AppData\Local\Temp\pip-req-build-e7hujacv'



  Cloning https://github.com/fmfn/BayesianOptimization to c:\users\cyberpower pc\appdata\local\temp\pip-req-build-e7hujacv
  Resolved https://github.com/fmfn/BayesianOptimization to commit 8ed5a3767634e5f47d0e3bf02b2dc11a6d241527
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


I use `bayesian-optimization==0.6`, my backend pretty much stick with this version, so migrating will break the code.

In [19]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
from bayes_opt import BayesianOptimization
sns.set()

In [20]:
import pkg_resources
import types


def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split('.')[0]
        elif isinstance(val, type):
            name = val.__module__.split('.')[0]
        poorly_named_packages = {'PIL': 'Pillow', 'sklearn': 'scikit-learn'}
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]
        yield name


imports = list(set(get_imports()))
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name != 'pip':
        requirements.append((m.project_name, m.version))

for r in requirements:
    print('{}=={}'.format(*r))

matplotlib==3.6.2
numpy==1.23.4
pandas==1.5.1
seaborn==0.12.1


In [21]:
def get_state(data, t, n):
    d = t - n + 1
    block = data[d : t + 1] if d >= 0 else -d * [data[0]] + data[0 : t + 1]
    res = []
    for i in range(n - 1):
        res.append(block[i + 1] - block[i])
    return np.array([res])

In [22]:
df = pd.read_csv('.\\Data\AMC.csv')
display(df.head())
display(df.tail())

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-01-02,7.30,7.56,7.23,7.46,7.414512,4545900
1,2020-01-03,7.45,7.60,7.31,7.32,7.275366,2218000
2,2020-01-06,7.25,7.30,7.11,7.12,7.076585,2903400
3,2020-01-07,7.11,7.20,7.00,7.13,7.086524,3205300
4,2020-01-08,7.11,7.15,6.61,6.62,6.579634,6557000


,Date,Open,High,Low,Close,Adj Close,Volume
713,2022-10-31,6.82,7.25,6.54,6.66,6.66,34859100
714,2022-11-01,6.76,6.85,6.13,6.15,6.15,21354300
715,2022-11-02,6.26,6.33,5.79,5.81,5.81,22112800
716,2022-11-03,5.75,5.85,5.58,5.70,5.70,16543200
717,2022-11-04,5.83,5.85,5.42,5.65,5.65,17995000


In [23]:
close = df.Close.values.tolist()
window_size = 30
skip = 5
l = len(close) - 1

In [24]:
class Deep_Evolution_Strategy:

    inputs = None

    def __init__(
        self, weights, reward_function, population_size, sigma, learning_rate
    ):
        self.weights = weights
        self.reward_function = reward_function
        self.population_size = population_size
        self.sigma = sigma
        self.learning_rate = learning_rate

    def _get_weight_from_population(self, weights, population):
        weights_population = []
        for index, i in enumerate(population):
            jittered = self.sigma * i
            weights_population.append(weights[index] + jittered)
        return weights_population

    def get_weights(self):
        return self.weights

    def train(self, epoch = 100, print_every = 1):
        lasttime = time.time()
        for i in range(epoch):
            population = []
            rewards = np.zeros(self.population_size)
            for k in range(self.population_size):
                x = []
                for w in self.weights:
                    x.append(np.random.randn(*w.shape))
                population.append(x)
            for k in range(self.population_size):
                weights_population = self._get_weight_from_population(
                    self.weights, population[k]
                )
                rewards[k] = self.reward_function(weights_population)
            rewards = (rewards - np.mean(rewards)) / np.std(rewards)
            for index, w in enumerate(self.weights):
                A = np.array([p[index] for p in population])
                self.weights[index] = (
                    w
                    + self.learning_rate
                    / (self.population_size * self.sigma)
                    * np.dot(A.T, rewards).T
                )
            if (i + 1) % print_every == 0:
                print(
                    'iter %d. reward: %f'
                    % (i + 1, self.reward_function(self.weights))
                )
        print('time taken to train:', time.time() - lasttime, 'seconds')


class Model:
    def __init__(self, input_size, layer_size, output_size):
        self.weights = [
            np.random.randn(input_size, layer_size),
            np.random.randn(layer_size, output_size),
            np.random.randn(layer_size, 1),
            np.random.randn(1, layer_size),
        ]

    def predict(self, inputs):
        feed = np.dot(inputs, self.weights[0]) + self.weights[-1]
        decision = np.dot(feed, self.weights[1])
        buy = np.dot(feed, self.weights[2])
        return decision, buy

    def get_weights(self):
        return self.weights

    def set_weights(self, weights):
        self.weights = weights

In [25]:
class Agent:
    def __init__(
        self,
        population_size,
        sigma,
        learning_rate,
        model,
        money,
        max_buy,
        max_sell,
        skip,
        window_size,
    ):
        self.window_size = window_size
        self.skip = skip
        self.POPULATION_SIZE = population_size
        self.SIGMA = sigma
        self.LEARNING_RATE = learning_rate
        self.model = model
        self.initial_money = money
        self.max_buy = max_buy
        self.max_sell = max_sell
        self.es = Deep_Evolution_Strategy(
            self.model.get_weights(),
            self.get_reward,
            self.POPULATION_SIZE,
            self.SIGMA,
            self.LEARNING_RATE,
        )

    def act(self, sequence):
        decision, buy = self.model.predict(np.array(sequence))
        return np.argmax(decision[0]), int(buy[0])

    def get_reward(self, weights):
        initial_money = self.initial_money
        starting_money = initial_money
        self.model.weights = weights
        state = get_state(close, 0, self.window_size + 1)
        inventory = []
        quantity = 0
        for t in range(0, l, self.skip):
            action, buy = self.act(state)
            next_state = get_state(close, t + 1, self.window_size + 1)
            if action == 1 and initial_money >= close[t]:
                if buy < 0:
                    buy = 1
                if buy > self.max_buy:
                    buy_units = self.max_buy
                else:
                    buy_units = buy
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
            elif action == 2 and len(inventory) > 0:
                if quantity > self.max_sell:
                    sell_units = self.max_sell
                else:
                    sell_units = quantity
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell

            state = next_state
        return ((initial_money - starting_money) / starting_money) * 100

    def fit(self, iterations, checkpoint):
        self.es.train(iterations, print_every = checkpoint)

    def buy(self):
        initial_money = self.initial_money
        state = get_state(close, 0, self.window_size + 1)
        starting_money = initial_money
        states_sell = []
        states_buy = []
        inventory = []
        quantity = 0
        for t in range(0, l, self.skip):
            action, buy = self.act(state)
            next_state = get_state(close, t + 1, self.window_size + 1)
            if action == 1 and initial_money >= close[t]:
                if buy < 0:
                    buy = 1
                if buy > self.max_buy:
                    buy_units = self.max_buy
                else:
                    buy_units = buy
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
                states_buy.append(t)
                print(
                    'day %d: buy %d units at price %f, total balance %f'
                    % (t, buy_units, total_buy, initial_money)
                )
            elif action == 2 and len(inventory) > 0:
                bought_price = inventory.pop(0)
                if quantity > self.max_sell:
                    sell_units = self.max_sell
                else:
                    sell_units = quantity
                if sell_units < 1:
                    continue
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell
                states_sell.append(t)
                try:
                    invest = ((total_sell - bought_price) / bought_price) * 100
                except:
                    invest = 0
                print(
                    'day %d, sell %d units at price %f, investment %f %%, total balance %f,'
                    % (t, sell_units, total_sell, invest, initial_money)
                )
            state = next_state

        invest = ((initial_money - starting_money) / starting_money) * 100
        print(
            '\ntotal gained %f, total investment %f %%'
            % (initial_money - starting_money, invest)
        )
        plt.figure(figsize = (20, 10))
        plt.plot(close, label = 'true close', c = 'g')
        plt.plot(
            close, 'X', label = 'predict buy', markevery = states_buy, c = 'b'
        )
        plt.plot(
            close, 'o', label = 'predict sell', markevery = states_sell, c = 'r'
        )
        plt.legend()
        plt.show()

In [26]:
def best_agent(
    window_size, skip, population_size, sigma, learning_rate, size_network
):
    model = Model(window_size, size_network, 3)
    agent = Agent(
        population_size,
        sigma,
        learning_rate,
        model,
        1000,
        5,
        5,
        skip,
        window_size,
    )
    try:
        agent.fit(100, 110)
        return agent.es.reward_function(agent.es.weights)
    except:
        return 0

In [27]:
def find_best_agent(
    window_size, skip, population_size, sigma, learning_rate, size_network
):
    global accbest
    param = {
        'window_size': int(np.around(window_size)),
        'skip': int(np.around(skip)),
        'population_size': int(np.around(population_size)),
        'sigma': max(min(sigma, 1), 0.0001),
        'learning_rate': max(min(learning_rate, 0.5), 0.000001),
        'size_network': int(np.around(size_network)),
    }
    print('\nSearch parameters %s' % (param))
    investment = best_agent(**param)
    print('stop after 100 iteration with investment %f' % (investment))
    if investment > accbest:
        costbest = investment
    return investment

In [28]:
accbest = 0.0
NN_BAYESIAN = BayesianOptimization(
    find_best_agent,
    {
        'window_size': (2, 50),
        'skip': (1, 15),
        'population_size': (1, 90),
        'sigma': (0.01, 0.99),
        'learning_rate': (0.00001, 0.49),
        'size_network': (10, 1000),
    },
)
NN_BAYESIAN.maximize(init_points = 30, n_iter = 500, acq = 'ei', xi = 0.0)

|   iter    |  target   | learni... | popula... |   sigma   | size_n... |   skip    | window... |
-------------------------------------------------------------------------------------------------

Search parameters {'window_size': 49, 'skip': 13, 'population_size': 65, 'sigma': 0.1423909483683146, 'learning_rate': 0.08422515468987396, 'size_network': 821}
time taken to train: 50.078895568847656 seconds
stop after 100 iteration with investment 249.579001
| 1         | 249.6     | 0.08423   | 65.15     | 0.1424    | 821.3     | 12.63     | 49.39     |

Search parameters {'window_size': 14, 'skip': 6, 'population_size': 48, 'sigma': 0.6849175878684723, 'learning_rate': 0.23077482341802796, 'size_network': 355}
time taken to train: 7.404277324676514 seconds
stop after 100 iteration with investment 310.363998
| 2         | 310.4     | 0.2308    | 47.86     | 0.6849    | 355.5     | 6.281     | 13.87     |

Search parameters {'window_size': 29, 'skip': 8, 'population_size': 3, 'sigma': 0.984

C:\Users\Cyberpower PC\AppData\Local\Temp\ipykernel_15324\1667285525.py:39: RuntimeWarning: invalid value encountered in divide
  rewards = (rewards - np.mean(rewards)) / np.std(rewards)


stop after 100 iteration with investment 0.000000
| 19        | 0.0       | 0.1909    | 8.804     | 0.08733   | 101.3     | 12.0      | 46.62     |

Search parameters {'window_size': 44, 'skip': 12, 'population_size': 61, 'sigma': 0.8026526801609339, 'learning_rate': 0.07050542695457092, 'size_network': 905}
time taken to train: 48.31713318824768 seconds
stop after 100 iteration with investment 146.704999
| 20        | 146.7     | 0.07051   | 61.32     | 0.8027    | 904.6     | 12.05     | 43.53     |

Search parameters {'window_size': 32, 'skip': 3, 'population_size': 80, 'sigma': 0.2022825665472919, 'learning_rate': 0.379585638704504, 'size_network': 343}
time taken to train: 227.9061577320099 seconds
stop after 100 iteration with investment 788.546996
| 21        | 788.5     | 0.3796    | 79.59     | 0.2023    | 343.1     | 3.169     | 32.27     |

Search parameters {'window_size': 15, 'skip': 4, 'population_size': 19, 'sigma': 0.44057604589285837, 'learning_rate': 0.307611899630872

C:\Users\Cyberpower PC\AppData\Local\Temp\ipykernel_15324\1667285525.py:39: RuntimeWarning: invalid value encountered in divide
  rewards = (rewards - np.mean(rewards)) / np.std(rewards)


stop after 100 iteration with investment 0.000000
| 37        | 0.0       | 0.363     | 82.41     | 0.01      | 336.6     | 5.895     | 34.06     |

Search parameters {'window_size': 37, 'skip': 2, 'population_size': 80, 'sigma': 0.23887082370199955, 'learning_rate': 0.1922256238945254, 'size_network': 346}
time taken to train: 320.439297914505 seconds
stop after 100 iteration with investment 864.675005
| 38        | 864.7     | 0.1922    | 80.07     | 0.2389    | 346.0     | 2.477     | 36.61     |

Search parameters {'window_size': 35, 'skip': 4, 'population_size': 83, 'sigma': 0.22319716077729287, 'learning_rate': 0.22334313745685969, 'size_network': 351}
time taken to train: 166.88320231437683 seconds
stop after 100 iteration with investment 555.773999
| 39        | 555.8     | 0.2233    | 82.55     | 0.2232    | 350.8     | 3.951     | 34.68     |

Search parameters {'window_size': 37, 'skip': 3, 'population_size': 81, 'sigma': 0.2692797501092664, 'learning_rate': 0.14901306587743

In [ ]:
print('Best AGENT accuracy value: %f' % NN_BAYESIAN.max['target'])
print('Best AGENT parameters: ', NN_BAYESIAN.max['params'])

#### My selected parameters

In [ ]:
best_agent(
    window_size = 30, 
    skip = 1, 
    population_size = 15, 
    sigma = 0.1, 
    learning_rate = 0.03, 
    size_network = 500
)

#### bayesian parameters

In [ ]:
best_agent(
    window_size = int(np.around(NN_BAYESIAN.max['params']['window_size'])), 
    skip = int(np.around(NN_BAYESIAN.max['params']['skip'])), 
    population_size = int(np.around(NN_BAYESIAN.max['params']['population_size'])), 
    sigma = NN_BAYESIAN.max['params']['sigma'], 
    learning_rate = NN_BAYESIAN.max['params']['learning_rate'], 
    size_network = int(np.around(NN_BAYESIAN.max['params']['size_network']))
)

#### My selected parameters

In [ ]:
model = Model(input_size = 30, 
              layer_size = 500, 
              output_size = 3)
agent = Agent(population_size = 15, 
              sigma = 0.1, 
              learning_rate = 0.03, 
              model = model, 
              money = 1000, 
              max_buy = 5, 
              max_sell = 5, 
              skip = 1, 
              window_size = 30)
agent.fit(500, 100)
agent.buy()

#### bayesian parameters

In [ ]:
model = Model(input_size = int(np.around(NN_BAYESIAN.max['params']['window_size'])), 
              layer_size = int(np.around(NN_BAYESIAN.max['params']['size_network'])), 
              output_size = 3)
agent = Agent(population_size = int(np.around(NN_BAYESIAN.max['params']['population_size'])), 
              sigma = NN_BAYESIAN.max['params']['sigma'], 
              learning_rate = NN_BAYESIAN.max['params']['learning_rate'], 
              model = model, 
              money = 1000, 
              max_buy = 5, 
              max_sell = 5, 
              skip = int(np.around(NN_BAYESIAN.max['params']['skip'])), 
              window_size = int(np.around(NN_BAYESIAN.max['params']['window_size'])))
agent.fit(500, 100)
agent.buy()



In [ ]:
from datetime import datetime
print(datetime.now())